In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_sheet_by_url(url, sheet = None):
  if sheet != None:
    data = gc.open_by_url(url).worksheet(sheet)
  else:
    data = gc.open_by_url(url).sheet1
  data = data.get_all_values() # get all values gives a list of rows
  data = pd.DataFrame.from_records(data) # convert to a DataFrame
  data = data.rename(columns=data.iloc[0]).drop(data.index[0]) # set column headers and drop the index
  return data

In [ ]:
def save_sheet(name, data, sheet = None):
  if sheet != None:
    ws = gc.open(name).worksheet(sheet)
  else:
    ws = gc.open(name).sheet1
  set_with_dataframe(ws, data)

In [ ]:
!pip install tabula-py

     |████████████████████████████████| 11.7MB 10.2MB/s 


In [ ]:
import tabula

In [ ]:
reports = load_sheet_by_url("https://docs.google.com/spreadsheets/d/1CB-nZ8QWcIZddDDaqP4TXmw5RiMiZyZQ08gQxVDVqbY/edit#gid=0", "feb 2")
reports.head()

,path,name
1,Adaire ABR.pdf,Adaire
2,"Allen, Ethan ABR.pdf","Allen, Ethan"
3,"Allen, Ethel ABR.pdf","Allen, Ethel"
4,Amy @ James Martin ABR.pdf,Amy @ James Martin
5,Anderson ABR.pdf,Anderson


In [ ]:
path_base = "/content/drive/Shareddrives/Data and Visuals Shared/Data Analysis/Bureaus/Philadelphia/Ventilation Reports/Reports Feb 2/"

In [ ]:
loop = reports.iloc[0:3,0]

In [ ]:
errors = []

In [ ]:
top = 121.43
left = 72.84
height = 612.8
width = 466.02
bottom = height + top
right = width + left
table_area = [top, left, bottom, right]

In [ ]:
ventilationA = pd.DataFrame()
ventilationB = pd.DataFrame()
ventilationC = pd.DataFrame()
ventilationD = pd.DataFrame()

for report in reports.itertuples():
    path = path_base + report.path

    tables = tabula.read_pdf(path, pages = "all", multiple_tables = False, lattice=True, area=table_area)
    table = tables[0]
    table["school"] = report.name
    print(report.name, len(table.columns))

    if  len(table.columns) == 10:
      table.columns = ["Classroom ID/Area ID Number","Classroom/Area Type supplied","Actual Outside Air CFM (OA)","Occupancy (OA CFM/15)","Window Fan Count","Window Fan Occupancy","Square Footage","Occupancy (SqFt/44)","Max Occupancy per Room","school"]
      ventilationA = pd.concat([ventilationA,table], ignore_index=True)
    elif  len(table.columns) == 9:
      table.columns = ["SDP Building Name","Classroom ID/Area ID Number","Classroom/Area Type supplied","Actual Outside Air CFM (OA)","Occupancy (OA CFM/15)","Classroom Square Footage","Occupancy (SqFt/44)","Max Occupancy per Room","school"]
      ventilationB = pd.concat([ventilationB,table], ignore_index=True)
    elif  len(table.columns) == 8:
      table.columns = ["Classroom ID/Area ID Number","Classroom/Area Type supplied","Actual Outside Air CFM (OA)","Occupancy (OA CFM/15)","Classroom Square Footage","Occupancy (SqFt/44)","Max Occupancy per Room","school"]
      ventilationC = pd.concat([ventilationC,table], ignore_index=True)
    else:
      table.columns = ["Classroom ID/Area ID Number","Classroom/Area Type supplied","Actual Outside Air CFM (OA)","Occupancy (OA CFM/15)","Classroom Square Footage","Occupancy (SqFt/44)","school"]
      ventilationD = pd.concat([ventilationD,table], ignore_index=True)
  

Adaire 8
Allen, Ethan 8
Allen, Ethel 8
Amy @ James Martin 8
Anderson 8
Arthur 8
Bache-Martin 8
Baldi 8
Barry 8
Bartram 8
Bethune 8
Blaine 8
Blankenburg 8
Bodine 8
Bregy 8
Bridesburg 10
Brown, HA 8
Brown, JH 8
Bryant 10
CAPA High 8
Carnell 8
Carver 8
Cassidy 8
Catharine 8
Central 8
Clemente 8
Cleveland 8
Comegys 8
Cook-Wissahickon 8
Cooke 8
Cramp 8
Crossan 8
Day 8
DeBurgos 8
Dick 8
Dobbins 8
Dobson 8
Duckrey 8
Dunbar 8
Edison 8
Edmonds, FS 8
Elverson 8
Emlen 8
Farrell 8
Fell 8
Feltonville A_S 8
Feltonville Intermediate 8
Finletter 8
Fitler 8
Fitzpatrick 8
Fox Chase _ Farm 8
Frank 8
Frankford 10
Franklin HS 8
Franklin Learning Center 8
Furness 8
GAMP 8
Gideon 10
Girard 8
Gompers 8
Greenberg 8
Greenfield 8
Hackett 8
Hamilton 8
Hancock 8
Harrington 8
Hartranft 8
Haverford Child Center 8
Heston 8
Hill-Freedman @ Leeds 8
Hopkinson 8
Howe 8
Hunter 8
Jackson 8
Jenks, AS 8
Jenks, J.S. 8
Juniata Park Academy 8
Kearny 8
Kelley, WD 8
Kelly, JB 8
Kenderton 8
Kensington CAPA 8
La Brum (Hancock) 8
La

In [ ]:
ventilationA

,Classroom ID/Area ID Number,Classroom/Area Type supplied,Actual Outside Air CFM (OA),Occupancy (OA CFM/15),Window Fan Count,Window Fan Occupancy,Square Footage,Occupancy (SqFt/44),Max Occupancy per Room,school
0,Main Building,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bridesburg
1,102,Classroom,422,28,NaN,NaN,846,19,1,Bridesburg
2,103,Classroom,377,25,NaN,NaN,686,15,1,Bridesburg
3,104,Classroom,476,31,NaN,NaN,993,22,2,Bridesburg
4,105,Classroom,300,20,NaN,NaN,672,15,1,Bridesburg
...,...,...,...,...,...,...,...,...,...,...
629,331,Classroom,NaN,NaN,1,18,700,15,NaN,Taylor
630,332,Classroom,NaN,NaN,1,18,750,17,NaN,Taylor
631,333,Classroom,NaN,NaN,1,18,675,15,NaN,Taylor
632,334,Classroom,NaN,NaN,1,18,625,14,NaN,Taylor


In [ ]:
ventilationB

,SDP Building Name,Classroom ID/Area ID Number,Classroom/Area Type supplied,Actual Outside Air CFM (OA),Occupancy (OA CFM/15),Classroom Square Footage,Occupancy (SqFt/44),Max Occupancy per Room,school
0,Building,103,Classroom,304,20,1130.000,25,NaN,Mastbaum
1,Building,104,Classroom,455,30,801.000,18,NaN,Mastbaum
2,Building,201,Classroom,0,0,NaN,0,NaN,Mastbaum
3,Building,202,Classroom,237,15,NaN,0,NaN,Mastbaum
4,Building,206,Classroom,569,37,1044.660,23,NaN,Mastbaum
...,...,...,...,...,...,...,...,...,...
81,Building,427-426,427-426,0,0,NaN,0,NaN,Mastbaum
82,Building,-,Gym,958,63,7555.890,171,NaN,Mastbaum
83,Building,Roof Mech. Rm,Gym Locker Rm.,0,0,NaN,0,NaN,Mastbaum
84,Building,Weld Shop,321 Nursing,614,40,1898.000,43,NaN,Mastbaum


In [ ]:
save_sheet("report_details_feb_2",ventilationA,"ventA")

In [ ]:
save_sheet("report_details_feb_2",ventilationB,"ventB")

In [ ]:
save_sheet("report_details_feb_2",ventilationC,"ventC")

In [ ]:
save_sheet("report_details_feb_2",ventilationD,"ventD")